### Run docker container

In [ ]:
!docker pull rafikrze/harmfulness_app:0.1
!docker run -p 8091:8000 -d rafikrze/harmfulness_app:0.1

### Load libraries and data

In [1]:
import requests
import json
import pandas as pd
import numpy as np
from sklearn import metrics
from tqdm.notebook import tqdm


text_test = pd.read_csv('https://storage.googleapis.com/poleval_data/test_set_only_text.txt',
                        sep="\n", header=None, names=['text'], encoding='utf-8')

url = 'http://localhost:8091/v1/predict/'
data_block = {"text": "to jest pierwszy test api"}
headers = {'token': 'test'}

## Measure time for same post data

In [2]:
%%timeit
requests.post(url, data=json.dumps(data_block), headers=headers)

29.5 ms ± 642 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Sample different data with API and measure time

In [4]:
%%timeit
data_block = {"text": text_test.sample(1).values[0][0]}
requests.post(url, data=json.dumps(data_block), headers=headers)

33.4 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Check metrics

In [5]:
# test / same functionality as in experiment
tag_test = pd.read_csv('https://storage.googleapis.com/poleval_data/test_set_only_tags.txt', sep="\n", header=None, names=['tag'], encoding='utf-8')

data_test = pd.concat([text_test, tag_test], axis=1)

# check data shape
assert(data_test.shape == (1000, 2))
print("The shape of the test data:", data_test.shape)

The shape of the test data: (1000, 2)


In [6]:
predictions_bin = list()
predictions_args = list()
real_value = list()
bad_text = list()

for index, row in tqdm(data_test.iterrows()):
    data_block = {"text": row['text']}
    r = requests.post(url, data=json.dumps(data_block), headers=headers)
    try:
        prediction = json.loads(r.content.decode('utf-8'))
    except Exception as e:
        # when api recognize bad language or to short twit continue
        bad_text.append(row)
        continue
    predictions_bin.append(int(prediction['prediction']))
    predictions_args.append(prediction['prediction_proba'])
    real_value.append(row['tag'])

In [7]:
# check binary and probability scores
assert all(predictions_bin == np.argmax(predictions_args, axis=1))

# Model Test Accuracy
print("Tested cases: ", len(real_value))
print("Logistic Regression Accuracy:",metrics.accuracy_score(real_value, predictions_bin))
print("Logistic Regression F1 micro:",metrics.f1_score(real_value, predictions_bin, average='micro'))
print("Logistic Regression F1 macro:",metrics.f1_score(real_value, predictions_bin, average='macro'))

Tested cases:  917
Logistic Regression Accuracy: 0.8669574700109052
Logistic Regression F1 micro: 0.8669574700109052
Logistic Regression F1 macro: 0.47482843778145795


In [8]:
#check what type of values were not included
pd.DataFrame(bad_text).groupby('tag').count()

,text
tag,
0,78
2,5
